<a href="https://colab.research.google.com/github/ngttgiang/company_matching/blob/master/Country_Latin_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install request

In [0]:
import requests
import lxml.html
import re
import pandas as pd
import string

In [0]:
#List of languages that use latin script
url = 'https://en.wikipedia.org/wiki/List_of_languages_by_writing_system'
html = requests.get(url)
doc = lxml.html.fromstring(html.content)
lang = [l.text_content() for l in doc.xpath('//*[@id="mw-content-text"]/div/ul[67]/li')]
lang = [re.sub('\[[a-z]+]|\[[0-9]+]|\\n|\(.+\)','',i).strip() for i in lang]

In [27]:
#List of countries and their languages
url2 = 'https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory'
html2 = requests.get(url2)
doc2 = lxml.html.fromstring(html2.content)
country = [i.text_content() for i in doc2.xpath('//table[@class = "wikitable"]//tr/td[1]')]
official_lang = [i.text_content() for i in doc2.xpath('//table[@class = "wikitable"]//tr/td[2]')]
national_lang = [i.text_content() for i in doc2.xpath('//table[@class = "wikitable"]//tr/td[5]')]

country = [re.sub('\[[a-z]+]|\[[0-9]+]|\\n|\(.+\)','',i).strip() for i in country]

official_lang = [re.sub('\[[a-z]+]|\[[0-9]+]|\(.+\)', '', i) for i in official_lang]
official_lang = [re.sub('\\n$|^\\n', '', i) for i in official_lang]
official_lang = [re.sub('\\n', ', ', i) for i in official_lang]

national_lang = [re.sub('\[[a-z]+]|\[[0-9]+]|\(.+\)', '', i) for i in national_lang]
national_lang = [re.sub('\\n$|^\\n', '', i) for i in national_lang]
national_lang = [re.sub('\\n', ', ', i) for i in national_lang]

data = pd.DataFrame()
data['Country'] = country
data['Lang'] = [ b if a == '' else a for a, b in zip(official_lang, national_lang)]
data['Lang2'] = [a.split(',') for a in data['Lang']]
data.head()

,Country,Lang,Lang2
0,Abkhazia,"Abkhaz, Russian","[Abkhaz, Russian]"
1,Afghanistan,"Pashto, Dari","[Pashto, Dari]"
2,Albania,Albanian,[Albanian]
3,Algeria,"Arabic, Tamazight","[Arabic, Tamazight]"
4,Andorra,Catalan,[Catalan]


In [0]:
#Flagging the country which uses Latin script

def find_language(lang_list, list_2_search):
  for l in lang_list:
    if list_2_search.count(l) >0:
      return('Y')
      break
  return('N')


data['Latin_Fl'] = [find_language(a, lang) for a in data['Lang2']]


In [24]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/Company_Matching")


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
data.to_csv('list.csv')